<a href="https://colab.research.google.com/github/reznimat/ppzd_sp/blob/main/semestralni_prace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Načtení knihoven**

In [23]:
import csv
import pandas as pd
import math as mt
import datetime as dt

**Načtení datových souborů**

In [24]:
# POZOR, pouze vzorek dat
url_sales = 'https://raw.githubusercontent.com/reznimat/ppzd_sp/main/SAMPLE_Online_Sales.csv'
#url_sales = 'https://raw.githubusercontent.com/reznimat/ppzd_sp/main/Online_Sales.csv'
df_sales = pd.read_csv(url_sales)

url_coupons = 'https://raw.githubusercontent.com/reznimat/ppzd_sp/main/Discount_Coupon.csv'
df_coupons = pd.read_csv(url_coupons,sep=";")

#dict_sales = df_sales.to_dict('records')

print('**Sales**')
print(df_sales.info())
print('\n------\n')
print('**Coupons**')
print(df_coupons.info())


**Sales**
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           30 non-null     int64  
 1   Transaction_ID       30 non-null     int64  
 2   Transaction_Date     30 non-null     object 
 3   Product_SKU          30 non-null     object 
 4   Product_Description  30 non-null     object 
 5   Product_Category     30 non-null     object 
 6   Quantity             30 non-null     int64  
 7   Avg_Price            30 non-null     float64
 8   Delivery_Charges     30 non-null     float64
 9   Coupon_Status        30 non-null     object 
dtypes: float64(2), int64(3), object(5)
memory usage: 2.5+ KB
None

------

**Coupons**
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0 

**Obohacení df_sales o atribut Month (Date Abrv)**

In [25]:
# Obohacení df_sales o Month pole
for i, row in df_sales.iterrows():
  Month = pd.to_datetime(row.Transaction_Date).strftime("%b")
  df_sales["Month"] = Month

print('**Sales**')
print(df_sales.info())

**Sales**
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           30 non-null     int64  
 1   Transaction_ID       30 non-null     int64  
 2   Transaction_Date     30 non-null     object 
 3   Product_SKU          30 non-null     object 
 4   Product_Description  30 non-null     object 
 5   Product_Category     30 non-null     object 
 6   Quantity             30 non-null     int64  
 7   Avg_Price            30 non-null     float64
 8   Delivery_Charges     30 non-null     float64
 9   Coupon_Status        30 non-null     object 
 10  Month                30 non-null     object 
dtypes: float64(2), int64(3), object(6)
memory usage: 2.7+ KB
None


**Spojení sales a coupons tabulek přes Product_Category a Month**

In [26]:
#Merge Sales a Coupons na základě Product Category a Month
df_sales_coupons = pd.merge(df_sales,df_coupons, on=["Product_Category","Month"])

print(df_sales_coupons.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           30 non-null     int64  
 1   Transaction_ID       30 non-null     int64  
 2   Transaction_Date     30 non-null     object 
 3   Product_SKU          30 non-null     object 
 4   Product_Description  30 non-null     object 
 5   Product_Category     30 non-null     object 
 6   Quantity             30 non-null     int64  
 7   Avg_Price            30 non-null     float64
 8   Delivery_Charges     30 non-null     float64
 9   Coupon_Status        30 non-null     object 
 10  Month                30 non-null     object 
 11  Coupon_Code          30 non-null     object 
 12  Discount_pct         30 non-null     int64  
dtypes: float64(2), int64(4), object(7)
memory usage: 3.2+ KB
None


**Počet a výpis jedinečných produktových kategorií**

In [ ]:
# Počet a výpis distinktních produktových kategorií
uniqPC = set()
for i, row in df_sales.iterrows():
  uniqPC.add(row["Product_Category"])

print("Produktových kategorií: "+ str(len(uniqPC)) )  # vypíše počet produktových kategorií
print(uniqPC)  # vypíše seznam produktových kategorií

Produktových kategorií: 6
{'Drinkware', 'Office', 'Nest-USA', 'Apparel', 'Lifestyle', 'Bags'}


**Suma tržeb pro kategorii Apparel**

In [ ]:
# Hint: InvoiceValue = (( QuantityAvg_price)(1-Dicount_pct)*(1+GST))+Delivery_Charges
# nejsou započítány náklady na dopravu a daně

ApparelSales = 0
AvgOrderPrice = 0

for i, row in df_sales_coupons.iterrows():
  if row.Product_Category == 'Apparel':
    if row.Coupon_Status =='Not Used':
      ApparelSales += (row.Quantity*row.Avg_Price) # Bez slevy
    else:
      ApparelSales += (row.Quantity*row.Avg_Price)*(1-(row.Discount_pct/100)) # Se slevou

print(ApparelSales)

191.77100000000002


In [ ]:
uniqMnth = set()
for i, row in df_sales.iterrows():
  uniqMnth.add(row["Month"])

print(uniqMnth)

{'Jan'}


**Tržby po kategoriích a měsících**

In [ ]:
my_list = [] #prázdný list

for kategorie in uniqPC:
  for mesic in uniqMnth:
    CategorySales = 0
    for i, row in df_sales_coupons.iterrows():
      if row.Product_Category == kategorie and row.Month == mesic:
        if row.Coupon_Status =='Not Used':
          CategorySales += (row.Quantity*row.Avg_Price) # Bez slevy
        else:
          CategorySales += (row.Quantity*row.Avg_Price)*(1-(row.Discount_pct/100)) # Se slevou
    my_list.append((kategorie, mesic, CategorySales)) # append vypočítaný záznam do listu
#    print()
#    slovnik["kategorie"] = kategorie
#    slovnik["mesic"] = mesic
#    slovnik["Sales"] = CategorySales

df_CatMonSales = pd.DataFrame.from_records(my_list, columns=['kategorie', 'mesic','Sales']) # převod listu na data_frame

print(df_CatMonSales)

   kategorie mesic    Sales
0  Drinkware   Jan  441.868
1     Office   Jan  181.779
2   Nest-USA   Jan  819.846
3    Apparel   Jan  191.771
4  Lifestyle   Jan    8.343
5       Bags   Jan   84.375
